# Automated Options Trading Monitor and Execution

This notebook automates the monitoring of option prices and places vertical spread orders when the conditions are met.

In [ ]:
import time
import logging
import datetime
import os
import sys

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

# Configuration
MAX_CYCLES = 5
CYCLE_WAIT_SECONDS = 60
MONITOR_RUNTIME_SECONDS = 60
DB_PATH = '../database/option_strategies.db'
IBKR_HOST = '127.0.0.1'
IBKR_PORT = 7497

# Market closed settings
ALLOW_MARKET_CLOSED_ORDERS = True  # Set to True to allow orders when market is closed

# Print startup message
logger.info("=" * 80)
logger.info("Starting automated trading system")
logger.info("=" * 80)

# Import the scripts directly
import price_monitor
import vertical_spread_order

cycle = 1
while cycle <= MAX_CYCLES:
    logger.info(f"Starting cycle {cycle}")
    
    # Step 1: Run price monitor
    logger.info("Starting price monitor...")
    try:
        price_monitor.run_price_monitor(
            db_path=DB_PATH,
            host=IBKR_HOST,
            port=IBKR_PORT,
            runtime_seconds=MONITOR_RUNTIME_SECONDS
        )
        logger.info("Price monitor completed successfully")
    except Exception as e:
        logger.error(f"Error running price monitor: {str(e)}")
    
    # Step 2: Place orders for triggered strategies
    logger.info("Starting order placement...")
    try:
        vertical_spread_order.run_trading_app(
            db_path=DB_PATH,
            target_date=datetime.datetime.now().strftime('%Y-%m-%d'),
            ibkr_host=IBKR_HOST,
            ibkr_port=IBKR_PORT,
            allow_market_closed=ALLOW_MARKET_CLOSED_ORDERS  # NEW PARAMETER
        )
        logger.info("Order placement completed successfully")
    except Exception as e:
        logger.error(f"Error placing orders: {str(e)}")
    
    if cycle < MAX_CYCLES:
        logger.info(f"Waiting {CYCLE_WAIT_SECONDS} seconds until next cycle...")
        time.sleep(CYCLE_WAIT_SECONDS)
    
    cycle += 1

logger.info("Automated trading system completed")

2025-04-11 12:41:24,642 - INFO - ================================================================================
2025-04-11 12:41:24,643 - INFO - Starting automated trading system
2025-04-11 12:41:24,644 - INFO - ================================================================================
2025-04-11 12:41:24,646 - INFO - Starting cycle 1
2025-04-11 12:41:24,647 - INFO - Starting price monitor...
2025-04-11 12:41:24,648 - ERROR - Error running price monitor: module 'price_monitor' has no attribute 'run_price_monitor'
2025-04-11 12:41:24,648 - INFO - Starting order placement...
2025-04-11 12:41:24,650 - INFO - Processing strategies for date: 2025-04-11
2025-04-11 12:41:24,650 - INFO - Market closed orders allowed: True
2025-04-11 12:41:24,653 - INFO - Getting strategies for date: 2025-04-11
2025-04-11 12:41:24,659 - INFO - Found 1 strategies to process
2025-04-11 12:41:24,677 - INFO - sent startApi
2025-04-11 12:41:24,679 - INFO - REQUEST startApi {}
2025-04-11 12:41:24,679 - INFO -

## Configure Market Closed Order Behavior

You can control whether orders are allowed to be placed when the market is closed by setting the `ALLOW_MARKET_CLOSED_ORDERS` variable at the top of the notebook.

- When `ALLOW_MARKET_CLOSED_ORDERS = True`: If the market is closed, orders will be placed using the estimated premium from the database.
- When `ALLOW_MARKET_CLOSED_ORDERS = False`: Orders will only be placed if the market is open AND the actual market premium is greater than or equal to the estimated premium.

This allows for more flexibility in strategy execution.